### 15 November 2019
# Bedtools slop RefSeq GRCh37 exonic regions
### Pavlos Bousounis
***Last updated 2019-11-17***

In [4]:
import glob
from natsort import natsorted
import numpy as np
import os
import pandas as pd
import pathlib
import pybedtools
from pybedtools import BedTool
import re

## Current working directory and date

In [3]:
basedir = '/Users/pbousounis/Experiments/2019-10-29_hg19mod/2019-11-10_RefSeq-ClinVar_GRCh37_slop_region_recovery'
os.chdir(basedir)
today = datetime.today().strftime('%Y-%m-%d')

print('\nCurrent working directory: {}\n'.format(os.getcwd()))

print('Today is: {}\n'.format(today))


Current working directory: /Users/pbousounis/Experiments/2019-10-29_hg19mod/2019-11-10_RefSeq-ClinVar_GRCh37_slop_region_recovery

Today is: 2019-11-17



In [ ]:
# list of slopped refseq bed files to intersect
slop_files = glob.glob('data/2019-11-13_RefSeqGRCh37_exon_slop_bed/*RefSeqGRCh37*slop*bed')

file = slop_files[1]

bedcols = ['chr', 'start', 'end', 'name']
df = pd.DataFrame(columns=bedcols)
for file in slop_files:

    # import file as dataframe
    tmp = pd.read_csv(file, sep='\t', header=None, names=bedcols)
        
    # extract transcript IDs and gene names to their own columns
    tmp['tx_id'] = tmp['name'].str.extract(r'(\w+-)(N(M|R)_\d+)')[1]
    #tmp['gene'] = tmp['name'].str.split('-').str[0]
    
    # add column with slop file ID
    tmp['slop_len'] = re.findall(r'slop\d+', file)[0]
    
    # split by transcript accession (RefSeq)
    tx_table = pd.DataFrame(columns=bedcols) 
    for tx in tmp.tx_id.unique():
        
        # subset by tx_id
        tmp_tx = tmp[tmp.tx_id == tx]
        
        # extract exon numbers (arbitrary; relative to transcript)
        tmp_tx.loc[:, 'exon_num'] = tmp_tx['name'].str.split('-', n=1).str[-1]
        
        # convert to bedtool 
        tx_bedtool = BedTool.from_dataframe(tmp_tx).sort()
        
        # bedtools merge -c 7 -o 'collapse'
        tx_merged = tx_bedtool.merge(c=7, o='collapse')
        
        # convert to dataframe
        tx_df = pd.read_table(tx_merged.fn, names=bedcols)
        
        # add tx_id column
        tx_df.loc[:, 'tx_id'] = tx_df['name'].str.split(':').str[0]

        # append to master table
        tx_table = pd.concat([tx_df, tx_table], sort=False)
        print('{} regions merged and added to master table.'.format(tx))
    
